In [ ]:
url = 'https://raw.githubusercontent.com/dranzerblaze/covid19worry/master/files_clean/data_incl_demographics.csv'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv(url)

In [ ]:
df.head()

,Unnamed: 0,V1,worry,chosen_emotion,anger,disgust,fear,anxiety,sadness,happiness,relaxation,desire,text_long,timing_textlong_firstclick,timing_textlong_lastclick,timing_textlong_submit,timing_textlong_nclicks,text_short,timing_textshort_firstclick,timing_textshort_lastclick,timing_textshort_submit,timing_textshort_nclicks,self_rating_general,self_rating_short,self_rating_long,twitter_general_often,twitter_tweet_often,twitter_participate_often,eng_native,ntok_long,nchar_long,ntok_short,nchar_short,cld_lang_long,cld_lang_short,id,n_punct,prop_punct,age,Country of Birth,Current Country of Residence,Employment Status,First Language,Nationality,Sex,Social-Media,Student Status
0,1,1,3,Sadness,5,5,3,7,7,2,4,5,It is less an much an issue of how it affects ...,4.340,150.638,202.320,12,It is very easy! Stay inside so you are not sp...,5.510,5.510,61.123,1,7,7,7,7,1,1,Yes,110,592,40,183,en,en,1815,9,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,8,Anxiety,6,7,7,8,6,4,3,1,I am concerned that the true impact of the cur...,3.924,615.838,621.392,24,"Things are difficult now, but we must all pull...",76.067,76.067,78.729,1,9,4,8,6,3,4,Yes,257,1424,43,221,en,en,401,16,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,4,Relaxation,1,1,2,2,4,7,7,2,"Personally, I am fairly calm about the corona ...",116.183,734.382,735.286,22,"People should try and remain calm, as panic wi...",1345.874,1345.874,1398.381,1,7,6,8,9,2,4,No but I speak it fluently,132,631,15,78,en,en,25,10,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,6,Relaxation,4,2,3,4,1,6,7,3,In this very moment as I am fortunate to be ab...,1.016,137.120,171.076,18,Fortunate to feel calm and relaxed but worried...,3.865,42.945,46.675,4,8,6,9,8,6,3,Yes,132,713,22,141,en,en,1585,12,0.02,28.0,Belgium,United Kingdom,Full-Time,English,France,Female,"Facebook, Youtube, Twitter, Linkedin",No
4,5,5,6,Anxiety,2,2,5,5,5,4,4,4,I am more worried about getting access to my n...,3.084,313.848,317.264,5,Not being able to cuddle my family sucks!,7.762,7.762,25.759,1,8,8,8,9,9,5,Yes,132,662,9,41,en,en,1207,13,0.02,57.0,United Kingdom,United Kingdom,"Not in paid work (e.g. homemaker', 'retired or...",English,United Kingdom,Female,Youtube,No


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def clean_text(text):
  text = text.lower()
  text = REPLACE_BY_SPACE_RE.sub(' ' , text)
  text = BAD_SYMBOLS_RE.sub(' ',text)
  clean_text = ' '
  for word in text.split():
    if word not in STOPWORDS:
      clean_text = clean_text + word + ' '
  return clean_text

In [ ]:
df['text_long'] = df['text_long'].apply(clean_text)

In [ ]:
df['text_long'] = df['text_long'].str.replace('\d+' ,' ')

In [ ]:
MAX_NB_WORDS = 2000
MAX_SEQUENCE_LENGTH = 500
EMBEDDING_DIM = 100

In [ ]:
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

Using TensorFlow backend.


In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['text_long'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 8239 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(df['text_long'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (2491, 500)


In [ ]:
Y = pd.get_dummies(df['chosen_emotion']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (2491, 8)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2241, 500) (2241, 8)
(250, 500) (250, 8)


In [ ]:
#LSTM Model

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(20))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          200000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 500, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               240800    
_________________________________________________________________
dense_1 (Dense)              (None, 20)                4020      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 168       
Total params: 444,988
Trainable params: 444,988
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 10
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2016 samples, validate on 225 samples
Epoch 1/10
2016/2016 [==============================] - 33s 16ms/step - loss: 1.6181 - accuracy: 0.5441 - val_loss: 1.3396 - val_accuracy: 0.5867
Epoch 2/10
2016/2016 [==============================] - 30s 15ms/step - loss: 1.3602 - accuracy: 0.5570 - val_loss: 1.3281 - val_accuracy: 0.5867
Epoch 3/10
2016/2016 [==============================] - 31s 15ms/step - loss: 1.3238 - accuracy: 0.5570 - val_loss: 1.3654 - val_accuracy: 0.5867
Epoch 4/10
2016/2016 [==============================] - 30s 15ms/step - loss: 1.2227 - accuracy: 0.5605 - val_loss: 1.4532 - val_accuracy: 0.5689
Epoch 5/10
2016/2016 [==============================] - 32s 16ms/step - loss: 1.1169 - accuracy: 0.6141 - val_loss: 1.5609 - val_accuracy: 0.5422


In [ ]:
# Getting the output of Layers 
from keras import backend as K

# with a Sequential model
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[3].output])

In [ ]:
layers_output = get_3rd_layer_output([X])

In [ ]:
layers_output[0][1][0]

0.13325037

In [ ]:
id = df['id']
dense_layer_output = []
df_dense_layer_output = pd.DataFrame()
for i in range(len(layers_output[0])):
  dense_layer_output.append(layers_output[0][i])

In [ ]:
df_dense_layer_output['id'] = id
df_dense_layer_output['output'] = dense_layer_output

In [ ]:
df_dense_layer_output.to_csv('dense_layer_output.csv')

In [ ]:
from sklearn.cluster import KMeans
kmeans  = KMeans(n_clusters = 8)

In [ ]:
x = df_dense_layer_output['output']
df_cluster = pd.DataFrame()
df_cluster['id'] = id
for i in range(20):
  j = i
  j = []
  for k in range(len(layers_output[0])):
    j.append(layers_output[0][k][i])
  df_cluster[str(i)] = j

In [ ]:
df_cluster.head()

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1815,0.046228,0.113866,0.045033,-0.010134,0.177952,0.311705,-0.840201,-0.161726,0.052286,0.363925,-0.652260,0.281591,-0.049353,-0.812854,-0.169597,-0.136133,-0.311472,-0.155448,-0.523279,-0.300819
1,401,0.133250,0.193932,0.168201,-0.113047,0.283664,0.326703,-1.145501,-0.016177,0.051451,0.385881,-0.802616,0.615754,-0.103530,-0.895792,-0.401284,-0.237747,-0.513856,-0.213717,-0.760715,-0.414253
2,25,0.018489,0.160838,-0.006908,-0.155278,0.047983,0.262422,-0.664082,-0.051694,0.145504,0.234295,-0.403626,0.372480,-0.180650,-0.572628,-0.211124,0.021070,-0.309687,-0.023232,-0.495942,-0.220811
3,1585,0.093484,0.228072,0.130458,-0.256968,0.157841,0.238932,-0.939505,0.053195,0.102249,0.230374,-0.573760,0.634606,-0.221357,-0.633546,-0.406645,-0.096473,-0.468270,-0.110303,-0.669549,-0.320240
4,1207,0.702737,0.423739,0.940426,-0.372415,1.061717,0.123969,-2.604922,0.490832,-0.468895,0.403781,-1.802593,1.640814,-0.080150,-1.248198,-1.266372,-0.999632,-1.194610,-0.829401,-1.507679,-1.002580


In [ ]:
x = df_cluster.drop(['id'], axis=1)

In [ ]:
kmeans.fit(x)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [ ]:
y_cluster = kmeans.predict(x)

In [ ]:
# Cluster Labels
df_labels = pd.DataFrame()
df_labels['id'] = id
df_labels['cluster'] = y_cluster
df_labels['emotion'] = df['chosen_emotion']

In [ ]:
df_labels.to_csv("cluster_labels.csv")

In [ ]:
# Finding dominant emotion in cluster
df_cluster_0 = df_labels[df_labels.cluster == 0]
df_cluster_0['emotion'].value_counts()

Anxiety       341
Sadness        43
Relaxation     34
Fear           18
Anger           8
Desire          2
Happiness       2
Disgust         1
Name: emotion, dtype: int64

In [ ]:
df_cluster_1 = df_labels[df_labels.cluster == 1]
df_cluster_1['emotion'].value_counts()

Relaxation    119
Anxiety        97
Sadness        89
Fear           71
Anger          21
Desire          5
Happiness       4
Disgust         1
Name: emotion, dtype: int64

In [ ]:
df_cluster_2 = df_labels[df_labels.cluster == 2]
df_cluster_2['emotion'].value_counts()

Anxiety       254
Sadness         5
Anger           4
Relaxation      3
Fear            2
Name: emotion, dtype: int64

In [ ]:
df_cluster_3 = df_labels[df_labels.cluster == 3]
df_cluster_3['emotion'].value_counts()

Anger         28
Fear          23
Sadness       19
Happiness     13
Relaxation    11
Disgust        7
Desire         6
Anxiety        4
Name: emotion, dtype: int64

In [ ]:
df_cluster_4 = df_labels[df_labels.cluster == 4]
df_cluster_4['emotion'].value_counts()

Anxiety    86
Sadness     3
Name: emotion, dtype: int64

In [ ]:
df_cluster_5 = df_labels[df_labels.cluster == 5]
df_cluster_5['emotion'].value_counts()

Anxiety       339
Relaxation     23
Sadness        20
Fear            9
Anger           6
Desire          2
Disgust         1
Name: emotion, dtype: int64

In [ ]:
df_cluster_6 = df_labels[df_labels.cluster == 6]
df_cluster_6['emotion'].value_counts()

Sadness       90
Fear          60
Relaxation    55
Anxiety       29
Anger         29
Happiness     17
Desire         9
Disgust        5
Name: emotion, dtype: int64

In [ ]:
df_cluster_7 = df_labels[df_labels.cluster == 7]
df_cluster_7['emotion'].value_counts()

Anxiety       231
Relaxation     88
Sadness        88
Fear           47
Anger          11
Desire          3
Happiness       3
Disgust         2
Name: emotion, dtype: int64

In [ ]:
df_dense_layer_output.head()

,id,output
0,1815,"[0.28202912, -0.10968205, -0.7513055, -0.42284..."
1,401,"[0.42700177, 0.061554946, -0.9832161, -0.59236..."
2,25,"[0.022476539, -0.18372938, -0.22511074, 0.0058..."
3,1585,"[0.088192925, -0.13339509, -0.4006941, -0.1378..."
4,1207,"[1.3047233, 0.75580734, -2.106079, -1.622969, ..."


In [ ]:
# GRU Model

from keras.layers import GRU
model2 = Sequential()
model2.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model2.add(SpatialDropout1D(0.2))
model2.add(GRU(200, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(8, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 100)          200000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 500, 100)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 200)               180600    
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 1608      
Total params: 382,208
Trainable params: 382,208
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 10
batch_size = 64

history = model2.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
# Bi - Directional LSTM
from keras.layers import Bidirectional
model3 = Sequential()
model3.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model3.add(SpatialDropout1D(0.2))
model3.add(Bidirectional(LSTM(64)))
model3.add(Dropout(0.5))
model3.add(Dense(8, activation='softmax'))
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model3.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 500, 100)          200000    
_________________________________________________________________
spatial_dropout1d_8 (Spatial (None, 500, 100)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               84480     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 1032      
Total params: 285,512
Trainable params: 285,512
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 5
batch_size = 64

history = model3.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train on 2016 samples, validate on 225 samples
Epoch 1/5
2016/2016 [==============================] - 78s 39ms/step - loss: 1.5020 - accuracy: 0.5570 - val_loss: 1.3274 - val_accuracy: 0.5867
Epoch 2/5
2016/2016 [==============================] - 78s 39ms/step - loss: 1.3952 - accuracy: 0.5570 - val_loss: 1.3244 - val_accuracy: 0.5867
Epoch 3/5
2016/2016 [==============================] - 77s 38ms/step - loss: 1.3898 - accuracy: 0.5570 - val_loss: 1.3343 - val_accuracy: 0.5867
Epoch 4/5
2016/2016 [==============================] - 78s 39ms/step - loss: 1.3719 - accuracy: 0.5570 - val_loss: 1.3189 - val_accuracy: 0.5867
Epoch 5/5
2016/2016 [==============================] - 78s 39ms/step - loss: 1.3500 - accuracy: 0.5570 - val_loss: 1.3246 - val_accuracy: 0.5867


In [ ]:
# Using  Un-Supervised Text Clustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [ ]:
# Total Corpus
document  = ''
for i in df['text_long']:
  document += i

In [ ]:
vectorizer = TfidfVectorizer(stop_words = 'english')

In [ ]:
x = vectorizer.fit_transform(df['text_long'])

In [ ]:
true_k = 8
model = KMeans(n_clusters = true_k , init = 'k-means++' , max_iter = 100 , n_init =1 )
model.fit(x)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=8, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [ ]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [ ]:
len(terms)

8054

In [ ]:
X = vectorizer.transform(["Covid 19 was very brutal with me"])
predicted = model.predict(X)
print(predicted)

[1]


In [ ]:
X = vectorizer.transform(["lockdown was  beneficial "])
predicted = model.predict(X)
print(predicted)

[1]


In [ ]:
# For short text
df['text_short'] = df['text_short'].apply(clean_text)
df['text_short'] = df['text_short'].str.replace('\d+' ,' ')


In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['text_short'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 4245 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(df['text_short'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (2491, 500)


In [ ]:
Y = pd.get_dummies(df['chosen_emotion']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (2491, 8)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2241, 500) (2241, 8)
(250, 500) (250, 8)


In [ ]:
#LSTM Model

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 100)          200000    
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 500, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               240800    
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 1608      
Total params: 442,408
Trainable params: 442,408
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 10
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2016 samples, validate on 225 samples
Epoch 1/10
2016/2016 [==============================] - 31s 15ms/step - loss: 1.5145 - accuracy: 0.5407 - val_loss: 1.3177 - val_accuracy: 0.5867
Epoch 2/10
2016/2016 [==============================] - 30s 15ms/step - loss: 1.3748 - accuracy: 0.5570 - val_loss: 1.3408 - val_accuracy: 0.5867
Epoch 3/10
2016/2016 [==============================] - 30s 15ms/step - loss: 1.3625 - accuracy: 0.5570 - val_loss: 1.3213 - val_accuracy: 0.5867
Epoch 4/10
2016/2016 [==============================] - 30s 15ms/step - loss: 1.3346 - accuracy: 0.5570 - val_loss: 1.3381 - val_accuracy: 0.5867


In [ ]:
accr = model.evaluate(X_test, Y_test)

250/250 [==============================] - 1s 2ms/step


In [ ]:
accr

[1.540956498146057, 0.5040000081062317]

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
ypred = model.predict(X_test)

In [ ]:
# Making Classes
for i in range(len(ypred)):
  max_pos = np.argmax(ypred[i])
  for j in range(len(ypred[i])):
    if j == max_pos:
      ypred[i][j] = 1
    else:
      ypred[i][j] = 0


In [ ]:
target_names= ['class0' , 'class1' , 'class2' , 'class3' ,'class4' , 'class5' , 'class6' , 'class7']

In [ ]:
print(classification_report(Y_test, ypred, target_names=target_names))

ValueError: ignored